# Extraction
В этом ноутбуке мы создадим pipeline, с помощью которого мы сможем выделять сущности из текста.

## Установка
### Зависимости
```bash
npm install --save langchain langchain-gigachat zod
```
### Переменные окружения
```bash
export GIGACHAT_CREDENTIALS='ваши креды'
export GIGACHAT_SCOPE='GIGACHAT_API_CORP' или 'GIGACHAT_API_PERS' или 'GIGACHAT_API_B2B'
```

## Схема
Здесь мы инициализируем схему, по которой GigaChat будет выделять сущности.
Схему мы инициализируем с помощью [Zod](https://zod.dev/)

In [8]:
import { z } from "zod";

const personSchema = z.object({
  name: z.optional(z.string()).describe("Имя человека"),
  hair_color: z
    .optional(z.string())
    .describe("Цвет волос, если известен"),
  height_in_meters: z
    .optional(z.number())
    .describe("Высота в метрах"),
});

## LLM цепочка


In [3]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

// Определите пользовательский промпт для предоставления инструкций и дополнительного контекста.
// 1) Вы можете добавить примеры в шаблон промпта, чтобы улучшить качество извлечения.
// 2) Введите дополнительные параметры, чтобы учитывать контекст (например, включите метаданные
//    о документе, из которого был извлечен текст.)
const promptTemplate = ChatPromptTemplate.fromMessages([
  [
    "system",
    `Ты алгоритм выделения сущностей из текста.
Выделяй только релевантную информацию из текста
Если ты не знаешь значение для атрибута, который нужно выделить проставь значение этого атрибута в null.`,
  ],
  ["human", "{text}"],
]);

In [9]:
import { Agent } from 'node:https';
import { GigaChat } from "langchain-gigachat"
/*
В обычном Node.js такое отключение проверки сертификатов срабатывает.

Но в Deno нет, 
поэтому используйте проставление NODE_EXTRA_CA_CERTS в env.
 */
const httpsAgent = new Agent({
    rejectUnauthorized: false,
}); 

const llm = new GigaChat({
    maxRetries: 0,
    model: 'GigaChat-Max',
    httpsAgent
})

In [10]:
const structured_llm = llm.withStructuredOutput(personSchema);

In [11]:
const prompt = await promptTemplate.invoke({
    text: "Алан Смит высотой 6 футов с белыми волосами ",
  });
await structured_llm.invoke(prompt);

UPDATE TOKEN


{ name: "Алан Смит", hair_color: "белый", height_in_meters: 1.8288 }